## 5.5 Metryki specyficzne dla klasyfikacji binarnej
Stworzyliśmy w poprzednim pliku prosty model który klasyfikował nam punkty na podstawie jednej tylko cechy. Byliśmy w stanie graficznie dostrzec jak model będzie klasyfikował nowe przykłady, ale w jaki sposób moglibyśmy sprawdzić model, gdyby cech było więcej? Musimy znać metody, które pozwolą nam zmierzyć jakość stworzonego systemu i dodatkowo będziemy mogli na ich podstawie wybrać najlepszy, jeśli akurat będziemy porównywać kilka różnych rozwiązań.

### Przykład klasyfikacji medycznej
Załóżmy, że tworzymy system, który ma na celu diagnozować pacjentów na podstawie ich wyników badań. Nie będzie to system ogólnego użytku, ale diagnozujący jedną konkretną chorobę, np. pewien specyficzny typ nowotworu. Klasą pozytywną będzie tutaj obecność tego schorzenia. Wyjątkowo, nie będziemy skupiać się na cechach i ich sposobach wykorzystania przez model, a jedynie stworzymy sobie pięć przykładów modeli, które będą popełniały innego rodzaju błędy i zobaczymy jak przełoży się to na wartość poszczególnych metryk.

Na potrzeby tego eksperymentu zakładamy, że mamy zbiór testowy opisujący 1000 pacjentów, a choroba dotyka 1,7% pacjentów, tj. w naszej próbie powinno być 17 takich osób. Skorzystamy jednak z generatora liczb pseudolosowych, aby wygenerować sobie takie dane.

In [2]:
import pandas as pd
import numpy as np

In [3]:
np.random.seed(63)

predictions_df = pd.DataFrame({
    "real_class": np.random.choice([0, 1], size=1000, p=[.983, .017])
})
predictions_df["real_class"].value_counts()

real_class
0    982
1     18
Name: count, dtype: int64

Musimy wprowadzić sobie kilka pojęć, które często będą przewijać się w pracy z metodami ML. Pod pojęciem confusion matrix (pol. tablica pomyłek) kryje się forma prezentacji predykcji dla modelu klasyfikacji binarnej.

In [1]:
from sklearn.metrics import confusion_matrix

### 1. Model zupełnie losowy
W naszym problemie mamy dwie klasy i podstawowym podejściem może być stworzenie modelu, który losowo zwraca wartość 0 lub 1, nie zwracając przy tym uwagi na rzeczywiste częstotliwości występowania.

In [4]:
predictions_df['pred_a'] = np.random.choice([0, 1], size=1000, p=[.5, .5])
predictions_df.sample(5)

,real_class,pred_a
370,0,0
688,0,1
403,0,0
726,0,1
921,0,0


In [6]:
random = confusion_matrix(predictions_df[["real_class"]], predictions_df['pred_a'])
random

array([[473, 509],
       [  8,  10]])

* 473 - liczba pacjentów których model stwierdził że nie są chorzy i faktycznie nie są
* 509 - liczba pacjentów których model stwierdził że są chorzy i a tak naprawdę nie są
* 8 - liczba pacjentów których model stwierdził że nie są chorzy a tak naprawdę są
* 10 - liczba pacjentów których model stwierdził że są chorzy i faktycznie są

### 2. Model losowy ważony
Znamy częstotliwość występowania choroby w populacji, więc możemy też stworzyć drugi model który będzie również losowo generował odpowiedzi, ale uwzględniając tę wartość.

In [7]:
predictions_df['pred_b'] = np.random.choice([0, 1], size=1000, p=[.983, .017])
predictions_df.sample(5)

,real_class,pred_a,pred_b
493,0,1,0
531,0,0,0
171,0,1,0
596,0,1,0
210,0,1,0


In [8]:
conf_matrix_b = confusion_matrix(predictions_df[["real_class"]], predictions_df["pred_b"])
conf_matrix_b

array([[966,  16],
       [ 17,   1]])

### 3. Model negatywny

In [9]:
predictions_df["pred_c"] = np.random.choice([0, 1], size=1000, p=[1, 0])
conf_matrix_c = confusion_matrix(predictions_df[["real_class"]], predictions_df["pred_c"])
conf_matrix_c

array([[982,   0],
       [ 18,   0]])

Skuteczność modelu na poziomie:

In [10]:
982/1000

0.982

Około 98,2% ;D

### 4. Model pozytywny

In [12]:
predictions_df["pred_d"] = np.random.choice([0, 1], size=1000, p=[0, 1])
conf_matrix_d = confusion_matrix(predictions_df[["real_class"]], predictions_df["pred_d"])
conf_matrix_d

array([[  0, 982],
       [  0,  18]])

Skuteczność na poziomie... 1,8% ;D

### 5. Model kontrolny
Stwórzmy też, poza wszystkimi poprzednimi przypadkami, jeden model kontrolny, który hipotetycznie mógłby zostać nauczony w projekcie ML. Nie będzie on działał idealnie, ale do jego wygenerowania wykorzystamy rzeczywiste klasy każdej z obserwacji.

In [14]:
predictions_df["pred_e"] = predictions_df["real_class"] \
    .where(np.random.random(size=1000) < 0.95, 1-predictions_df["real_class"])
predictions_df.sample(5)

,real_class,pred_a,pred_b,pred_c,pred_d,pred_e
253,0,1,0,0,1,0
987,1,0,0,0,1,1
397,0,1,0,0,1,0
994,0,1,0,0,1,0
950,0,0,0,0,1,0


predictions_df[["real_class"]] <- to jest dataframe, predictions_df["real_class"] <- to jest data Series

In [22]:
conf_matrix_e = confusion_matrix(predictions_df["real_class"], predictions_df["pred_e"])
conf_matrix_e

array([[934,  48],
       [  0,  18]])

In [23]:
model_columns = ["a", "b", "c", "d", "e"]

### Przegląd metryk klasyfikacji binarnej

##### Accuracy
Jest to najprostsza z metryk i bardzo intuicyjna, stąd też dość powszechna. Definiuje się ją jako stosunek prawidłowych predykcji, w stosunku do wszystkich predyckji.
accuracy = TP + TN / TP + FP + FN + TP

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
for model in model_columns:
    print(
        f"Model {model}:",
        accuracy_score(predictions_df["real_class"], predictions_df[f"pred_{model}"])
    )

Model a: 0.483
Model b: 0.967
Model c: 0.982
Model d: 0.018
Model e: 0.952


Okazuje się, że model który jest zawsze negatywny, ma największą wydajność liczoną za pomocą metryki accuracy. Metryki tej używamy głównie dla zbalansowanych zbiorów i wtedy, gdy chcemy być w stanie wyjaśnić wszystkim jak dobry jest nasz model.

#### **Precision**
Miara ta wyznacza jak wiele obserwacji oznaczonych jako pozytywne, jest rzeczywiście pozytywne. W przypadku diagnozy naszej choroby, jest to stosunek liczby pacjentów poprawnie zdiagnozowanych jako chorzy, w stosunku do liczby wszystkich osób dla których system przewidział klasę pozytywną.
* precision = TP / TP + FP

In [26]:
from sklearn.metrics import precision_score

In [28]:
for model in model_columns:
    print(f"Model {model}: ",
          precision_score(predictions_df["real_class"], predictions_df[f"pred_{model}"]))

Model a:  0.019267822736030827
Model b:  0.058823529411764705
Model c:  0.0
Model d:  0.018
Model e:  0.2727272727272727


/home/maciej/deep/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Recall / Sensivity
Jeśli chcielibyśmy obliczyć jak wiele elementów rzeczywiście pozytywnych zostało przez nas poprawnie zaklasyfikowane jako pozytywne, to recall jest odpowiednią metryką. W naszym przykładzie, zależałoby nam na wyłapaniu każdego chorego, ale nie zwracamy uwagi na to czy zdrowych pacjentów zaklasyfikujemy również jako chorych.
* recall = TP / TP + FN

In [29]:
from sklearn.metrics import recall_score

In [30]:
for model in model_columns:
    print(f"Model {model}: ",
          recall_score(predictions_df["real_class"], predictions_df[f"pred_{model}"]))

Model a:  0.5555555555555556
Model b:  0.05555555555555555
Model c:  0.0
Model d:  1.0
Model e:  1.0


#### F-Score
Metryki precision i recall mogą być naszym celem optymalizacji również łącznie. Jak porównać dwa modele na podstawie dwóch wartości? F-Score jest odpowiedzią, ponieważ łączy obie metryki.
* f score = (1 + Beta)^2 precision * recall / Beta^2 * precision + recall
* Za pomocą parametru Beta możemy sterować wagami każdej z metryki, im wyższa wartość B tym bardziej dbamy o optymalizacje metryki recall. Typowo stosuje się metrykę *F1* oraz *F2*, gdzie parametr B jest ustawiony kolejno na 1 i 2. 

In [31]:
from sklearn.metrics import fbeta_score

In [33]:
for model in model_columns:
    print(f"Model {model}: ",
          fbeta_score(predictions_df["real_class"], predictions_df[f"pred_{model}"],beta = 1))

Model a:  0.037243947858473
Model b:  0.05714285714285714
Model c:  0.0
Model d:  0.03536345776031434
Model e:  0.42857142857142855


### Szukanie optymalnego modelu
Znajomość problemu może zasugerować nam na jakiego rodzaju błędy możemy się zgodzić. Machine learning jest w stanie stworzyć modele, które będą najczęściej poprawne, jednak nie będą one perfekcyjne. Wybór odpowiedniej metryki pozwala wybrać model, który popełnia głównie nieszkodliwe dla nas pomyłki.

In [34]:
from sklearn.metrics import classification_report

In [36]:
for model in model_columns:
    print(f"Model {model}: \n",
          classification_report(predictions_df["real_class"], predictions_df[f"pred_{model}"]))

Model a: 
               precision    recall  f1-score   support

           0       0.98      0.48      0.65       982
           1       0.02      0.56      0.04        18

    accuracy                           0.48      1000
   macro avg       0.50      0.52      0.34      1000
weighted avg       0.97      0.48      0.64      1000

Model b: 
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       982
           1       0.06      0.06      0.06        18

    accuracy                           0.97      1000
   macro avg       0.52      0.52      0.52      1000
weighted avg       0.97      0.97      0.97      1000

Model c: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       982
           1       0.00      0.00      0.00        18

    accuracy                           0.98      1000
   macro avg       0.49      0.50      0.50      1000
weighted avg       0.96      0.98      0.

/home/maciej/deep/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maciej/deep/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maciej/deep/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/maciej/deep/lib/python